In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
def get_holidays(years):
    holidays = list()
    for year in years:
        holidays_wrong = pd.read_json('https://api.calendario.com.br/?json=true&ano=%s&ibge=2304400&token=bWFkc29uZGRpYXNAZ21haWwuY29tJmhhc2g9MTIzNDc0NTU0' % year).date.astype('str').values
        holidays.extend(['%s-%s-%s' % (x.split('-')[0], x.split('-')[2], x.split('-')[1]) for x in holidays_wrong])
    return holidays

In [3]:
path_name = '../data/climate/'
file_names_pr = ['24279', '24319', '24320', '24321']
file_names_t  = ['225', '572']

df_final = pd.DataFrame()

for file_name in file_names_pr:
    df_ = pd.read_csv(path_name+'pr/'+file_name+'.csv')
    df_final[file_name] = df_['pr']

for file_name in file_names_t:
    df_final['avg_tmp_'+file_name] = pd.read_csv(path_name+'t2m/'+file_name+'.csv')['avg_tmp']
    df_final['max_tmp_'+file_name] = pd.read_csv(path_name+'t2mmax/'+file_name+'.csv')['max_tmp']
    df_final['min_tmp_'+file_name] = pd.read_csv(path_name+'t2mmin/'+file_name+'.csv')['min_tmp']
    
df_['year']  = df_['year'].astype('object')
df_['month'] = df_['month'].astype('object')
df_['day']   = df_['day'].astype('object')
df_final = df_final.assign(weekday=df_.apply(lambda x: datetime.strptime('%s-%s-%s' % (x['year'],x['month'],x['day']),'%Y-%m-%d').weekday(), axis=1),
                           year=df_['year'],
                           month=df_['month'],
                           day=df_['day'])

holidays_list = get_holidays(['2014','2015','2016', '2017'])
df_final = df_final.assign(holiday=df_.apply(lambda x: 1 if '%s-%s-%s' % (str(x['year']), str(x['month']).zfill(2),str(x['day']).zfill(2)) in holidays_list else 0, axis=1))
df_final = df_final.assign(workingday=df_final.apply(lambda x: 1 if x['holiday']==1 or x['weekday'] in [5,6] else 0, axis=1))
df_final = df_final.assign(avg_pr=df_final[file_names_pr].mean(axis=1))
df_final = df_final.assign(max_pr=df_final[file_names_pr].max(axis=1))
df_final = df_final.assign(min_pr=df_final[file_names_pr].min(axis=1))
df_final = df_final.assign(avg_tmp=df_final[['avg_tmp_225','avg_tmp_572']].mean(axis=1))
df_final = df_final.assign(min_tmp=df_final[['min_tmp_225','min_tmp_572']].mean(axis=1))
df_final = df_final.assign(max_tmp=df_final[['max_tmp_225','max_tmp_572']].mean(axis=1))
df_final = df_final.drop(['avg_tmp_225','avg_tmp_572','min_tmp_225','min_tmp_572','max_tmp_225','max_tmp_572'], axis=1)
df_final = df_final.drop(file_names_pr, axis=1)

In [4]:
df_final

,day,month,weekday,year,holiday,workingday,avg_pr,max_pr,min_pr,avg_tmp,min_tmp,max_tmp
0,1,1,2,2014,1,1,0.000,0.0,0.0,27.8835,25.05,31.45
1,2,1,3,2014,0,0,0.750,3.0,0.0,27.2055,25.00,29.85
2,3,1,4,2014,0,0,2.350,6.0,0.0,26.6630,23.90,30.70
3,4,1,5,2014,0,1,0.000,0.0,0.0,27.2940,24.15,31.20
4,5,1,6,2014,0,1,1.000,4.0,0.0,27.3610,24.65,31.30
5,6,1,0,2014,0,0,0.000,0.0,0.0,26.9275,24.95,30.35
6,7,1,1,2014,0,0,0.500,2.0,0.0,27.3160,24.10,30.70
7,8,1,2,2014,0,0,0.000,0.0,0.0,27.7585,24.45,31.65
8,9,1,3,2014,0,0,0.000,0.0,0.0,27.4620,24.40,31.75
9,10,1,4,2014,0,0,0.000,0.0,0.0,27.8615,24.35,32.05


In [12]:
df_final

,avg_tmp_225,max_tmp_225,min_tmp_225,avg_tmp_572,max_tmp_572,min_tmp_572,day,month,weekday,year,holiday,workingday,avg_pr,max_pr,min_pr
0,27.50,30.4,25.1,28.267,32.5,25.0,1,1,2,2014,1,1,0.000,0.0,0.0
1,26.94,29.4,25.1,27.471,30.3,24.9,2,1,3,2014,0,0,0.750,3.0,0.0
2,26.48,30.7,24.3,26.846,NaN,23.5,3,1,4,2014,0,0,2.350,6.0,0.0
3,27.60,31.6,24.1,26.988,30.8,24.2,4,1,5,2014,0,1,0.000,0.0,0.0
4,27.38,30.6,24.6,27.342,32.0,24.7,5,1,6,2014,0,1,1.000,4.0,0.0
5,26.28,29.7,25.1,27.575,31.0,24.8,6,1,0,2014,0,0,0.000,0.0,0.0
6,27.74,31.6,24.1,26.892,29.8,24.1,7,1,1,2014,0,0,0.500,2.0,0.0
7,27.70,31.6,24.5,27.817,31.7,24.4,8,1,2,2014,0,0,0.000,0.0,0.0
8,27.42,31.7,24.1,27.504,31.8,24.7,9,1,3,2014,0,0,0.000,0.0,0.0
9,28.04,32.2,24.0,27.683,31.9,24.7,10,1,4,2014,0,0,0.000,0.0,0.0


In [29]:
3/4

0.75

In [114]:
holidays_list

['2014-01-01',
 '2014-03-03',
 '2014-04-03',
 '2014-05-03',
 '2014-03-19',
 '2014-03-19',
 '2014-03-25',
 '2014-01-04',
 '2014-04-13',
 '2014-04-18',
 '2014-04-18',
 '2014-04-20',
 '2014-04-21',
 '2014-01-05',
 '2014-11-05',
 '2014-12-06',
 '2014-06-19',
 '2014-06-19',
 '2014-10-08',
 '2014-08-15',
 '2014-07-09',
 '2014-12-10',
 '2014-10-15',
 '2014-10-17',
 '2014-10-28',
 '2014-02-11',
 '2014-11-15',
 '2014-11-20',
 '2014-12-25',
 '2015-01-01',
 '2015-02-16',
 '2015-02-17',
 '2015-02-18',
 '2015-03-19',
 '2015-03-19',
 '2015-03-25',
 '2015-01-04',
 '2015-03-04',
 '2015-03-04',
 '2015-05-04',
 '2015-04-13',
 '2015-04-21',
 '2015-01-05',
 '2015-10-05',
 '2015-04-06',
 '2015-04-06',
 '2015-12-06',
 '2015-09-08',
 '2015-08-15',
 '2015-07-09',
 '2015-12-10',
 '2015-10-15',
 '2015-10-17',
 '2015-10-28',
 '2015-02-11',
 '2015-11-15',
 '2015-11-20',
 '2015-12-25',
 '2016-01-01',
 '2016-08-02',
 '2016-09-02',
 '2016-10-02',
 '2016-03-19',
 '2016-03-19',
 '2016-03-25',
 '2016-03-25',
 '2016-03-